In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
from utilities import *
from models import *

import joblib

In [3]:
ble = read_ble('../Data/ble.csv')

In [4]:
coor = None

In [5]:
def reg_impute(data):
    for s in scanners:
        dx = coor[0] - ble[s]['X']
        dy = coor[1] - ble[s]['Y']
        d = np.sqrt(np.square(dy) + np.square(dx))
        reg = joblib.load('../Models/linear_distance_models/{}.joblib'.format(s))
        ss = pd.Series(reg.predict(d.reshape((-1, 1))))
        data[s].fillna(ss, inplace=True)

In [6]:
model = MLPRegressor(model_type='special')
model.model.fit(np.array([[0] * 17]), np.array([[0, 0]]))
model.model.load_weights('../Models/MLP_Regression_Rolling_MinMax_LinearDistanceModel.h5')

Train on 1 samples
1/1 [==============================] - 5s 5s/sample - loss: 0.6642 - mae: 0.0000e+00 - mse: 0.0000e+00


In [7]:
client = InfluxDBClient()

In [8]:
data = client.retrieveData(seconds=3, beacon="0117C55D14E4")

In [9]:
data = format_data(data)

In [10]:
data[scanners] = min_max_scaling(data[scanners])

In [11]:
data = data.rolling(15, min_periods=1).mean().reset_index()

In [12]:
data

,index,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
0,2019-12-26 00:58:42.382000+00:00,NaN,0.266667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-26 00:58:42.386000+00:00,NaN,0.266667,NaN,NaN,NaN,0.65,NaN,NaN,NaN,NaN,NaN,NaN,0.550000,NaN,NaN,NaN,NaN
2,2019-12-26 00:58:42.427000+00:00,NaN,0.266667,0.483333,NaN,NaN,0.65,NaN,NaN,NaN,NaN,NaN,NaN,0.550000,NaN,NaN,NaN,NaN
3,2019-12-26 00:58:42.428000+00:00,NaN,0.266667,0.483333,NaN,NaN,0.65,0.450000,NaN,NaN,NaN,NaN,NaN,0.550000,NaN,NaN,NaN,NaN
4,2019-12-26 00:58:42.457000+00:00,NaN,0.266667,0.483333,NaN,NaN,0.65,0.450000,0.166667,NaN,NaN,NaN,NaN,0.550000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2019-12-26 00:58:45.267000+00:00,NaN,0.400000,0.416667,0.3,NaN,0.55,0.466667,NaN,0.4,NaN,0.425,0.5,0.561111,0.1,0.333333,NaN,0.183333
100,2019-12-26 00:58:45.282000+00:00,NaN,0.400000,0.416667,NaN,NaN,0.55,0.466667,0.083333,0.4,NaN,0.425,0.5,0.561111,0.1,0.333333,NaN,0.183333
101,2019-12-26 00:58:45.297000+00:00,NaN,0.400000,0.416667,NaN,NaN,0.55,0.466667,0.083333,0.4,0.15,0.400,0.5,0.561111,0.1,0.333333,NaN,0.183333
102,2019-12-26 00:58:45.298000+00:00,NaN,0.400000,0.416667,NaN,NaN,0.55,0.466667,0.191667,0.4,0.15,0.400,0.5,0.541667,0.1,0.333333,NaN,0.183333


In [13]:
if coor is None:
    data = data.fillna(0)
else:
    reg_impute(data)

In [14]:
coor = model.predict(data[scanners]).mean(axis=0)

In [15]:
np.expand_dims(coor, axis=0)

array([[22.163961 , 11.7283945]], dtype=float32)

In [16]:
coor

array([22.163961 , 11.7283945], dtype=float32)

In [17]:
data

,index,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
0,2019-12-26 00:58:42.382000+00:00,0.0,0.266667,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,2019-12-26 00:58:42.386000+00:00,0.0,0.266667,0.000000,0.0,0.0,0.65,0.000000,0.000000,0.0,0.00,0.000,0.0,0.550000,0.0,0.000000,0.0,0.000000
2,2019-12-26 00:58:42.427000+00:00,0.0,0.266667,0.483333,0.0,0.0,0.65,0.000000,0.000000,0.0,0.00,0.000,0.0,0.550000,0.0,0.000000,0.0,0.000000
3,2019-12-26 00:58:42.428000+00:00,0.0,0.266667,0.483333,0.0,0.0,0.65,0.450000,0.000000,0.0,0.00,0.000,0.0,0.550000,0.0,0.000000,0.0,0.000000
4,2019-12-26 00:58:42.457000+00:00,0.0,0.266667,0.483333,0.0,0.0,0.65,0.450000,0.166667,0.0,0.00,0.000,0.0,0.550000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2019-12-26 00:58:45.267000+00:00,0.0,0.400000,0.416667,0.3,0.0,0.55,0.466667,0.000000,0.4,0.00,0.425,0.5,0.561111,0.1,0.333333,0.0,0.183333
100,2019-12-26 00:58:45.282000+00:00,0.0,0.400000,0.416667,0.0,0.0,0.55,0.466667,0.083333,0.4,0.00,0.425,0.5,0.561111,0.1,0.333333,0.0,0.183333
101,2019-12-26 00:58:45.297000+00:00,0.0,0.400000,0.416667,0.0,0.0,0.55,0.466667,0.083333,0.4,0.15,0.400,0.5,0.561111,0.1,0.333333,0.0,0.183333
102,2019-12-26 00:58:45.298000+00:00,0.0,0.400000,0.416667,0.0,0.0,0.55,0.466667,0.191667,0.4,0.15,0.400,0.5,0.541667,0.1,0.333333,0.0,0.183333


In [18]:
if coor is None:
    data = data.fillna(0)
else:
    reg_impute(data)

In [19]:
coor = model.predict(data[scanners]).mean(axis=0)

In [20]:
coor

array([22.163961 , 11.7283945], dtype=float32)